In [2]:
import spacy
import utils

from collections import defaultdict
from joblib import Parallel, delayed
from multiprocessing import Manager
from tqdm import tqdm

from spacy.tokens import Doc

In [3]:
corpus = utils.read_file("/home/km55359/rawdata/babylm_data/babylm_100M/sents/babylm_sents.txt")

In [4]:
model = spacy.load("en_core_web_sm")

In [5]:
# batch into groups of 1k
# batch_size = 6250
# batches = []
# batch = []
# for line in mini_corpus:
#     if batch_size < 0:
#         batches.append(line)
#     else:
#         batch.append(line)
#         if len(batch) == batch_size:
#             batches.append(batch)
#             batch = []
# if batch:
#     batches.append(batch)

# def get_children_recursive(token):
#     children = []
#     for child in token.children:
#         children.append({
#             'text': child.text,
#             'dep': child.dep_,
#             'children': get_children_recursive(child)
#         })
#     return children

def get_children_flatten(token, depth=0, dep=False):
    children = []
    for child in token.children:
        if dep:
            children.append((child.text.lower(), child.dep_, depth, child.i))
        else:
            children.append(child.text.lower())
        children.extend(get_children_flatten(child, depth+1, dep))
    return children

In [6]:
# mini_corpus = corpus[:100000]
idx = 0
docs = model.pipe(corpus, batch_size=2048)
indef_articles_with_pl_nouns = []
for doc in tqdm(docs):
    for token in doc:
        if token.tag_ == "NNS" or token.tag_ == "NNPS":
            children = get_children_flatten(token, dep=True)
            if len(children) >= 1:
                # children_toks, children_deps = list(zip(*children))
                # if "a" in children_toks or "an" in children_toks or "another" in children_toks:
                #     found = True
                #     indef_articles_with_pl_nouns.append((idx, token.text, children_toks, children_deps))
                #     break
                for child in children:
                    # check for specific determiners and the fact that their indices are less than that of the noun:
                    if child[0] in ["a", "an", "another"] and child[3] < token.i:
                        found = True
                        indef_articles_with_pl_nouns.append(
                            (
                                idx,
                                token.text,
                                token.i,
                                child[0],
                                child[1],
                                child[2],
                                child[3],
                            )
                        )
                        break
            else:
                pass

            # for child in token.children:
            # if child.text in ["a", "an", "another"]:
            #     found = True
            #     indef_articles_with_pl_nouns.append((idx, token.text, child.dep_, child.text))
            #     break
    idx += 1

22528it [00:21, 1059.03it/s]


KeyboardInterrupt: 

In [1]:
len(indef_articles_with_pl_nouns)

NameError: name 'indef_articles_with_pl_nouns' is not defined

In [134]:
indef_articles_with_pl_nouns

# write to csv
import csv
with open("../data/babylm-analysis/indef_articles_with_pl_nouns.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence_idx", "noun", "noun_idx", "article", "article_dep", "dep_depth", "article_idx"])
    for row in indef_articles_with_pl_nouns:
        writer.writerow(row)

In [74]:
# mini_corpus = [
#     "A few days should be enough for that task!",
#     "A hundred dollars is a lot of money.",
#     "A few hundred dollars is all we need for this mission.",
#     "The hundred dollars I spent ended up being a lot of money.",
#     "Five days is a long time to wait for a response.",
#     "Seven weeks of thesis writing takes a toll on one's body.",
#     "A year is a long time to wait.",
#     "Five people is a not a huge number of attendees.",
#     "Revenue exceeded twelve billion dollars, with a loss of $1b.",
#     "Net income was $9.4 million compared to the prior year of $2.7 million.",
#     "Five people is a not a huge number of attendees.",
#     "A dozen eggs is too many.",
#     "A dozen eggs was too many.",
#     "All it took was seven days.",
#     "He was going to the bathroom, and all he took was seven minutes.",
#     "7 days is what it took for him to get over it.",
#     "He takes 7 days to get over it.",
#     "That will take another thousand days.",
#     "A dozen eggs was too many.",
#     "The family spent a beautiful five days in austin."
# ]
# idx = 0
# indef_articles_with_pl_nouns = []
# for sentence in tqdm(mini_corpus):
#     doc = model(sentence)
#     found = False
#     children = []
#     for token in doc:
#         if token.tag_ == "NNS" or token.tag_ == "NNPS":
#             # children = get_children_flatten(token)
#             # print(token.text, children)
#             children = get_children_flatten(token, dep=True)
#             # print(children)
#             if len(children) >= 1:
#                 children_toks, children_deps = list(zip(*children))
#                 print(token.text, children_toks, children_deps)
#             else:
#                 pass

#             # X = token
#             # for child in token.children:
#             #     if child.text in ["a", "an", "another"]:
#             #         found = True
#             #         indef_articles_with_pl_nouns.append((idx, token.text, child.dep_, child.text))
#             #         break

#     if found:
#         print(sentence)
#     idx+=1


100%|██████████| 20/20 [00:00<00:00, 293.85it/s]

days ('a', 'few') ('det', 'amod')
dollars ('hundred', 'a') ('nummod', 'quantmod')
dollars ('hundred', 'a', 'few') ('nummod', 'quantmod', 'amod')
dollars ('the', 'hundred', 'spent', 'i') ('det', 'nummod', 'relcl', 'nsubj')
days ('five',) ('nummod',)
weeks ('seven', 'of', 'writing', 'thesis') ('nummod', 'prep', 'pobj', 'compound')
people ('five',) ('nummod',)
dollars ('billion', 'twelve') ('nummod', 'compound')
people ('five',) ('nummod',)
eggs ('dozen', 'a') ('nummod', 'quantmod')
eggs ('dozen', 'a') ('nummod', 'quantmod')
days ('seven',) ('nummod',)
minutes ('seven',) ('nummod',)
days ('7',) ('nummod',)
days ('7',) ('nummod',)
days ('another', 'thousand') ('det', 'nummod')
eggs ('dozen', 'a') ('nummod', 'quantmod')
days ('a', 'beautiful', 'five') ('det', 'amod', 'nummod')
